**NOTE: This notebook has no github workflows as the motive of this notebook is to provide a solution how to use private packages in AML SDK v2.**

# Create Azure Machine Learning Custom Environment

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace - [Configure workspace](../../jobs/configuration.ipynb) 
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create a custom environment with private packages from python SDK using a docker image

**Motivations** - Azure Machine Learning environments are an encapsulation of the environment where your machine learning training happens. By default your workspace has several curated environments already available. This notebook explains how to create a custom environment to run your specific task if you need to.

# 1. Define Your Custom Docker Image
Create a `Dockerfile` that installs your private wheel. For example:

```dockerfile
FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04

# Install pip and wheel
RUN apt-get update && apt-get install -y python3-pip
RUN pip3 install wheel

# Install your private wheel
COPY your-package.whl /tmp/
RUN pip3 install /tmp/your-package.whl
```

Build and push this image to your Azure Container Registry (ACR):​

```bash
docker build -t <your-acr-name>.azurecr.io/your-image:latest .
docker push <your-acr-name>.azurecr.io/your-image:latest
```
Ensure Azure ML has access to your ACR


# 2. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 2.1. Import the required libraries

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment
from azure.identity import DefaultAzureCredential

## 2.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

OR

In [ ]:
ml_client = MLClient.from_config(DefaultAzureCredential())

# 3. Create Environment
Azure Machine Learning [environments](https://docs.microsoft.com/en-us/azure/machine-learning/concept-environments) are an encapsulation of the environment where your machine learning training happens. They specify the Python packages, environment variables, and software settings around your training and scoring scripts. They also specify run times (Python, Spark, or Docker). The environments are managed and versioned entities within your Machine Learning workspace that enable reproducible, auditable, and portable machine learning workflows across a variety of computes.

The workspace contains several curated environments by default to use as-is. However, you can create your own custom environment to meet your specific needs.

The `Environment` class will be used to create a custom environment. It accepts the following key parameters:
- `name` - Name of the environment.		
- `version`	- Version of the environment. If omitted, Azure ML will autogenerate a version.		
- `image` - The Docker image to use for the environment. Either `image` or `build` is required to create environment.
- `conda_file` - The standard conda YAML [configuration file](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-file-manually) of the dependencies for a conda environment. It can be used with a `image`. If specified, Azure ML will build the conda environment on top of the Docker image provided.
- `BuildContext`- The Docker build context configuration to use for the environment. Either `image` or `build` is required to create environment.
  - `path`- Local path to the directory to use as the build context.		
  - `dockerfile_path` - Relative path to the Dockerfile within the build context.
- `description`	- Description of the environment.		

## 3.1 Create a custom environment using a docker image
In this sample we will create an environment in Azure Machine Learning workspace using a docker image. The `Environment` is used to initialize the details. The `MLClient` is used to create the environment in the workspace.


In [ ]:
env = Environment(
    name="custom-docker-env",
    image="<your-acr-name>.azurecr.io/your-image:latest",
    description="Environment created from a custom Docker image.",
)

# Set the base image to a custom Docker image
env = ml_client.environments.create_or_update(env)

# 3. Define a job within the custom environment

In [ ]:
from azure.ai.ml import load_job


job = ml_client.jobs.create_or_update(
    name="docker_job",
    env="custom-docker-env",
    job=load_job(
        "../tests/test_configs/command_job/command_job_test_local_env.yml",
        params_override=[{"name": "docker_job"}, {"compute": "kchawla1"}],
    ),
)